In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='3' 

In [2]:
import fastai
import ffmpeg
from fastai import *
from fastai.vision import *
from fastai.callbacks.tensorboard import *
from fastai.vision.gan import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.loss import *
from fasterai.filters import *
from fasterai.generators import *
from pathlib import Path
from itertools import repeat
from IPython.display import HTML, display
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True

In [3]:
#Adjust render_factor (int) if image doesn't look quite right (max 64 on 11GB GPU).  The default here works for most photos.  
#It literally just is a number multiplied by 16 to get the square render resolution.  
#Note that this doesn't affect the resolution of the final output- the output is the same resolution as the input.
#Example:  render_factor=21 => color is rendered at 16x21 = 336x336 px.  
render_factor=43

#Specify media_url. Many sources will work (YouTube, Imgur, Twitter, Reddit, etc). 
#Complete list here: https://rg3.github.io/youtube-dl/supportedsites.html . 
#NOTE:  Make source_url None to just read from file at ./video/source/[file_name] directly without modification
#source_url = 'https://vimeo.com/87890004'
#source_url = 'https://www.youtube.com/watch?v=gZShc8oshtU'
#source_url = 'https://www.youtube.com/watch?v=fk6qiJjEEBo'
#source_url = 'https://twitter.com/silentmoviegifs/status/1088830101863759872'
source_url = None
file_name = 'video14.mp4'

In [4]:
#colorizer = get_video_colorizer(render_factor=render_factor)
colorizer = get_video_colorizer2(render_factor=render_factor)

/media/jason/Projects/Deep Learning/DeOldifyV2/DeOldify/fastai/data_block.py:426: UserWarning: Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.")
/media/jason/Projects/Deep Learning/DeOldifyV2/DeOldify/fastai/data_block.py:429: UserWarning: Your validation set is empty. If this is by design, use `no_split()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


In [5]:
if source_url is not None:
    colorizer.colorize_from_url(source_url, file_name)
else:
    colorizer.colorize_from_file_name(file_name)

KeyboardInterrupt: 